In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import json
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import scale
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.metrics import matthews_corrcoef, roc_auc_score, average_precision_score

from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score,confusion_matrix

In [3]:
def res_to_csv(res, bacteria, mode, encoding, method, drug_name):
    tp = []
    for k in res.keys():
        if k != 'accuracy':
            tp.append(list(res[k].values()))
        else:
            tp.append([np.nan, np.nan, res[k], res['macro avg']['support']])
    tp = pd.DataFrame(tp, index=res.keys(), columns=res["S"].keys())
    tp.to_csv(f"results/{bacteria}/{encoding}/{bacteria}_{mode}_{encoding}_{method}_{drug_name}.csv", index=True, encoding='utf-8')


In [4]:
def MLPredModel(X, Y, bacteria, mode, encoding, method, drug_name, Normalization=True,seed=7, save_res='easy'):

    MODEL = {
        'LR': LogisticRegression(),
        'RF': RandomForestClassifier(),
        'SVM': SVC()
    }
    if encoding == 'FCGR':
        X = X.reshape(X.shape[0], -1)
    if Normalization:
        X = scale(X)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
    sample_solver= BorderlineSMOTE()
    sample_solver=EditedNearestNeighbours()
    x_train,y_train=sample_solver.fit_resample(x_train,y_train)
    model = MODEL[method]

    param_grid = {"LR":{
                      'C': [0.5, 1, 2],
                      'max_iter':[5000]
                  },
                  "RF":{
                      'n_estimators': [50, 100],
                      'max_depth': [None, 10, 20],
                      'min_samples_split': [2, 5, 10],
                      'min_samples_leaf': [1, 2, 4],
                      'bootstrap': [True, False]
                  },
                  "SVM":{
                      'C': [ 1, 3, 5],
                      'kernel': ['linear', 'sigmoid'],
                      'gamma': ['scale', 'auto']
                  }
    }
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    grid_search = GridSearchCV(model, param_grid[method], cv=kf, scoring='accuracy')
    print(f"{bacteria},{drug_name},Training {method} model ...")
#     print(len(x_train))

    grid_search.fit(x_train, y_train)
    # plot_feature_importance(model, bacteria, mode, encoding, method, drug_name, POS)
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    preds = best_model.predict(x_test)
    cm = confusion_matrix(y_test, preds)

    MCC = matthews_corrcoef(y_test, preds)
    AUROC = roc_auc_score(y_test, preds)
    AUPRC = average_precision_score(y_test, preds)
    precision = precision_score(y_test, preds, average='macro')
    recall = recall_score(y_test, preds, average='macro')
    f1 = f1_score(y_test, preds, average='macro')
    acc = accuracy_score(y_test, preds)
    print(f"{bacteria},{drug_name},Training {method} model success")

    return [MCC, AUROC, AUPRC,precision, recall, f1, acc],cm

In [5]:
def plot_feature_importance(model, bacteria, mode, encoding, method, drug_name, POS, n=10):
    if method == 'RF':
        index = np.argsort(-model.feature_importances_)[:n].tolist()
        value = model.feature_importances_[index]
    if method == 'LR':
        index=np.argsort(-abs(model.coef_[0]))[:n].tolist()
        value = model.coef_[0][index]
        
    print(f"Most {n} important position: \n{POS[index]}")
    fig, ax = plt.subplots()
    ax.barh(range(n), value, align='center', color='c')
    ax.set_yticks(range(n))
    ax.set_yticklabels([str(i) for i in POS[index]])
    ax.invert_yaxis()
    plt.savefig(f'results/{bacteria}/{encoding}/{mode}_{method}_{drug_name}_TOP{n}_feature_importance.png')
    plt.show()
    plt.close()


In [6]:
Methods = ['LR','RF','SVM']
# Methods = ['RF']
# Encodings = ['FCGR',]
Encoding = 'Label_Encoding'
Drug_list = ['AMP', 'AMX', 'AMC', 'TZP', 'CXM', 'CET', 'TBM', 'TMP', 'CIP', 'CTX', 'CTZ', 'GEN']
# Drug_list = ['AMX']
Bacteria = 'E.coli'
Mode = 'ToN'
save_res='easy'
seed = [7,15,27,34,54,65,79,89,97,105]


In [7]:
for m in Methods:
    # if m != "LR":
    #     continue
    res_all_drug=[]
    for d in Drug_list:
        res = []
        data = np.load(f'data/{Bacteria}/preprocessed/{Encoding}/{Bacteria}_{Mode}_{Encoding}_{d}.npz', allow_pickle=True)
        X, Y, POS = data['X'].astype('float32'), data['Y'].astype('int32'), data['POS'].astype('int32')
        for sd in seed:
            r,cm = MLPredModel(X, Y, bacteria=Bacteria, mode=Mode,encoding=Encoding, method=m, drug_name=d,Normalization=False,seed=sd)
            res.append(r)
            if not os.path.exists(f'result1220/{Bacteria}/{Encoding}/'):
                os.makedirs(f'result1220/{Bacteria}/{Encoding}/')
            df = pd.DataFrame(cm, index=['S', 'R'], columns=['S', 'R'])
            df.to_csv(f'result1220/{Bacteria}/{Encoding}/{Mode}_{m}_{d}_seed{sd}_cm.csv')
        res_mean = np.mean(np.array(res), axis=0)
        if save_res=='easy':
            res_mean_df = pd.DataFrame([res_mean],columns=['MCC','AUROC','AUPRC','Precision', 'Recall', 'F1-score','acc'])

            res_mean_df.to_csv(f'result1220/{Bacteria}/{Encoding}/{Mode}_{m}_{d}_result.csv')
        res_all_drug.append(res_mean.tolist())
    res_all_drug = pd.DataFrame(res_all_drug,columns=['MCC','AUROC','AUPRC','Precision', 'Recall', 'F1-score','accuracy'],index=Drug_list)

    res_all_drug.to_csv(f'result1220/{Bacteria}/{Encoding}_all_drug_{m}_{d}_result.csv')


In [9]:
def get_POSweight(drug_name,sd):
    np.random.seed(sd)#CTX
    # np.random.seed(0)#CTZ
    # np.random.seed(7)

    data = np.load(f'data/{Bacteria}/preprocessed/{Encoding}/{Bacteria}_{Mode}_{Encoding}_{drug_name}.npz', allow_pickle=True)
    X, Y, POS = data['X'].astype('float32'), data['Y'].astype('int32'), data['POS'].astype('int32')
    shuffe_idx=list(range(len(X)))
    np.random.shuffle(shuffe_idx)
    X1=X[shuffe_idx]
    Y1=Y[shuffe_idx]
    model=RandomForestClassifier(n_estimators=200, random_state=0)
    model.fit(X1,Y1)
    index = np.argsort(-model.feature_importances_).tolist()
    print(f"TOP 10 POS: {POS[index][:10]}")
    with open(f'results/E.coli/Label_Encoding/{drug_name}_POS_weight.json', 'w', encoding='utf-8') as f:
        json.dump(model.feature_importances_.tolist(), f, ensure_ascii=False, indent=4)
    


    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
    model=RandomForestClassifier(n_estimators=200, random_state=0)
    model.fit(x_train,y_train)
    preds = model.predict(x_test)
    print("raw Result for {}".format(drug_name))
    print(classification_report(y_test, preds, target_names=['S', 'R']))
    # res1=classification_report(y_test, preds, target_names=['S', 'R'],output_dict=True)
    X2=X[:,index[:50]]
    X2=(X2!=4)
    x_train, x_test, y_train, y_test = train_test_split(X2, Y, test_size=0.2, random_state=seed)
    model=RandomForestClassifier(n_estimators=200, random_state=0)
    model.fit(x_train,y_train)
    preds = model.predict(x_test)
    print("POS Result for {}".format(drug_name))
    print(classification_report(y_test, preds, target_names=['S', 'R']))
    # res2=classification_report(y_test, preds, target_names=['S', 'R'],output_dict=True)



    # X2=X[:,index[:50]]
    # X2=(X2!=4)
    # x_train, x_test, y_train, y_test = train_test_split(X2, Y, test_size=0.2, random_state=seed)
    # model=LogisticRegression(solver='lbfgs', max_iter=1500)
    # model.fit(x_train,y_train)
    # preds = model.predict(x_test)
    # print("Result for {}".format(drug_name))
    # print(classification_report(y_test, preds, target_names=['S', 'R']))
    # return res1['accuracy'],res2['accuracy']

In [ ]:
get_POSweight('AMX',1)

In [ ]:
# res={'CTX':[],'CTZ':[]}
# for d in ['CTX','CTZ']:
#     for sd in range(100):
#         res1,res2=get_POSweight(d,sd)
#         if res2>res1:
#             print(d+f'  {res1}  {res2}')
#             res[d].append({'seed':sd,'res1':res1,'res2':res2})
#         if len(res[d])>3: break
# print(res)

In [1]:
# for d in Drug_list:
#     get_POSweight(d)

## 验证关键位置的突变与耐药性的相关性

In [16]:
# snp = pd.read_csv('/data/HWK/DeepGene/data/E.coli/preprocessed/FCGR/E.coli_ToN_FCGR_input.csv', sep=',', encoding='utf-8')
# snp

In [17]:
# snp=snp.set_index(data['POS'])
# snp

In [18]:
# plt.bar(range(len(data['Y'])),data['Y'])

In [19]:
# imp_pos=POS[index][:10]
# tmp=(snp.loc[imp_pos[0]]!='N')
# plt.bar(range(len(tmp)),tmp)

In [20]:
# np.corrcoef(data['Y'],tmp.to_numpy())